# Коллаборативная фильтрация

- библиотека SURPRISE

- датасет MovieLens 1M

- цель: RMSE на тестовой выборке 0.87 и ниже



In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# устанавливаем библиотеку surprise

!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095427 sha256=e4d418ff7fdbe8a94048fab7189d2c69974d186e917a434fcab4ad758aae163a
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
# импортируем необходимые методы

from surprise import KNNWithMeans, KNNBasic, KNNWithZScore
from surprise import SlopeOne, CoClustering
from surprise import NMF
from surprise import SVDpp
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

В датасете MovieLens на один миллион записей файлы оказались в формате dat.

Прочтем их в pandas как csv-файлы, указав разделитель "::" и предварительно подготовив список названий столбцов.

In [ ]:
movies_columns = ['movieId', 'title', 'genres']
movies = pd.read_csv('movies.dat', sep='::', encoding='latin-1', names=movies_columns)


<ipython-input-5-2128e23f822c>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv('movies.dat', sep='::', encoding='latin-1', names=movies_columns)


In [ ]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [ ]:
ratings_columns = ['userId', 'movieId', 'rating', 'timestamp']
ratings = pd.read_csv('ratings.dat',sep='::', names=ratings_columns)

<ipython-input-7-75e01bf219e6>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv('ratings.dat',sep='::', names=ratings_columns)


In [ ]:
ratings

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [ ]:
# количество пользователей и распределение оценок

ratings.userId.value_counts()

4169    2314
1680    1850
4277    1743
1941    1595
1181    1521
        ... 
5725      20
3407      20
1664      20
4419      20
3021      20
Name: userId, Length: 6040, dtype: int64

In [ ]:
# количество фильмов и распределение оценок

ratings.movieId.value_counts()

2858    3428
260     2991
1196    2990
1210    2883
480     2672
        ... 
3458       1
2226       1
1815       1
398        1
2909       1
Name: movieId, Length: 3706, dtype: int64

In [ ]:
# объединим датафреймы

df = ratings.merge(movies, on='movieId')
df

,userId,movieId,rating,timestamp,title,genres
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
1000204,5949,2198,5,958846401,Modulations (1998),Documentary
1000205,5675,2703,3,976029116,Broken Vessels (1998),Drama
1000206,5780,2845,1,958153068,White Boys (1999),Drama
1000207,5851,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western


In [ ]:
# проверим наличие пропущенных значений

df.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
title        0
genres       0
dtype: int64

In [ ]:
# создадим специальный датафрейм для библиотеки surprise

dataset = pd.DataFrame({
    'uid': df.userId,
    'iid': df.title,
    'rating': df.rating
})

In [ ]:
# размах оценок рейтинга

ratings.rating.max(), ratings.rating.min()

(5, 1)

In [ ]:
# загружаем датасет, задаем шкалу оценок

reader = Reader(rating_scale=(1.0, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [ ]:
# разобьем наобучающую и тестовую выборки, фиксируя случайность

trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
# количество уникальных пользователей и уникальных фильмов

dataset['uid'].nunique(), dataset['iid'].nunique()

(6040, 3706)

Поскольку количество пользователей почти в два раза превышает количество фильмов, будем использовать item-based рекомендации.

# Модель "k ближайших соседей со средними" на 50 соседей с косинусным расстоянием


In [ ]:
# алгоритм "k ближайших соседей со средними" на 50 соседей с косинусным расстоянием

algo = KNNWithMeans(k=50, sim_options={
    'name': 'cosine',
    'user_based': False  # compute  similarities between ~users
})
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
test_pred= algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8903


0.8903359903312722

На всякий случай, убедимся, что аналогичный подход user-based работает в данном случае хуже:


In [ ]:
algo_ = KNNWithMeans(k=50, sim_options={
    'name': 'cosine',
    'user_based': True
})
algo_.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
test_pred_= algo_.test(testset)
accuracy.rmse(test_pred_, verbose=True)

RMSE: 0.9343


0.9343199767489837

# Модель "k ближайших соседей со стандартизацией" на 30 соседей с косинусным расстоянием


In [ ]:
algo2 = KNNWithZScore(k=30, sim_options={
    'name': 'cosine',
    'user_based': False
})
algo2.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
test_pred2= algo2.test(testset)
accuracy.rmse(test_pred2, verbose=True)

RMSE: 0.8941


0.8940835562051966

# Модель SlopeOne

In [ ]:
algo3 = SlopeOne()
algo3.fit(trainset)

In [ ]:
test_pred3= algo3.test(testset)
accuracy.rmse(test_pred3, verbose=True)

RMSE: 0.9043


0.9042559248430544

# Модель CoClustering

In [ ]:
algo4 = CoClustering()
algo4.fit(trainset)

In [ ]:
test_pred4= algo4.test(testset)
accuracy.rmse(test_pred4, verbose=True)

RMSE: 0.9128


0.9128419059796228

# Модель NMF. "Неотрицательная факторизация матриц"

In [ ]:
algo5 = NMF()
algo5.fit(trainset)

In [ ]:
test_pred5= algo5.test(testset)
accuracy.rmse(test_pred5, verbose=True)

RMSE: 0.9155


0.9155082890578301

In [ ]:
# увеличим число факторов и количество эпох в этой же модели

algo5_ = NMF(n_factors=100, n_epochs=50, random_state=42)
algo5_.fit(trainset)

In [ ]:
test_pred5_= algo5_.test(testset)
accuracy.rmse(test_pred5_, verbose=True)

RMSE: 1.1244


1.1244483755017292

# Модель SVDpp. "Сингулярное разложение матриц плюс-плюс"

In [ ]:
algo6 = SVDpp()
algo6.fit(trainset)

In [ ]:
test_pred6= algo6.test(testset)
accuracy.rmse(test_pred6, verbose=True)

RMSE: 0.8598


0.8598470387664905

Таким образом, модель SVDpp с дефолтными настройками обеспечила наилучший результат (RMSE = 0.8598), который соответствует постановке задачи (RMSE = 0.87 и ниже). Обучение этой модели заняло максимальное количество времени по сравнению со всеми использованными моделями.